# Stationary Emission 

## Overview


This notebook walks us through to Calculate emissions(Stationary) from comfort heating or combustion of fuels in stationary combustion sources.

Scope 1 Emissions are calculation of Direct greenhouse gas (GHG) emissions that can be directly controlled by an organization. The source of the emissions comes from sources that are owned by the organization, for example, fossil fuel.

Stationary Emisssion is part of Scope 1 Emissions.


### Setup

Make sure you have Python installed in your system (python 3+ is required).


<b>Note:</b> To run this notebook seamlessly, you have to first configure your GHG Emission APIs' credentials in a file named `secrets.ini` in the format below:

```
[EI]
api.api_key = <Your GHG APIs API key>
api.client_id = <Your GHG APIs client Id>





```

Keep the secrets.ini file at an appropriate relative location of this notebook. For example, as specified in the config below.

```config.read('../../auth/secrets.ini')```

In [7]:
# Install the packages below using pip/pip3 based on your python version.
# pip install pandas,configparser,json,IPython
import pandas as pd
import configparser
import requests
import json
from IPython.display import display as display_summary

#### Get Token

Run the below code snippet to generate the Auth Bearer Token using your api_key configured in secrets.ini.

In [14]:
config = configparser.RawConfigParser()
config.read(['../../auth/secrets.ini','../../auth/config.ini'])

EI_API_KEY  = config.get('EI', 'api.api_key')
EI_CLIENT_ID = config.get('EI', 'api.client_id')
EI_AUTH_ENPOINT = config.get('EI', 'api.auth_endpoint')
EI_BASE_URL = config.get('EI', 'api.base_url')

def get_bearer_token(token):
    headers = {
        'Content-Type': 'application/json',
        'cache-control': 'no-cache',
    }
    data = {"apiKey":token, "clientId":EI_CLIENT_ID}
    bearer_token_req = requests.post(EI_AUTH_ENPOINT, headers=headers, data= json.dumps(data))
    if bearer_token_req.status_code != 200:
        print( "Error in getting Bearer token. Error code :",bearer_token_req.status_code)
    return bearer_token_req.json()['access_token']

mybearer_token = get_bearer_token(EI_API_KEY)


The following code snippet set the statoinary Emission Parmeters

### Utility Functions

#### Function to call API

In [ ]:
def Call_Carbon_API(CO2API, Co2Data, token):
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer '+token,
        'Content-Type': 'application/json',
    }

    CO2response_req = requests.post( CO2API, headers=headers, data=json.dumps(Co2Data))
    if CO2response_req.status_code != 200:
        print( "Problem! Error in generating response. Error code : ",CO2response_req.status_code)
    return CO2response_req.json()

#### Input formats for API

In [ ]:
def customer_id(customerid):
    Input =  {
              "id": customerid
          }
    return Input 

def customer_onBehalfOfClient(companyId,companyName):
    Input =  {
              "companyId": companyId,
              "companyName": companyName
          }
    return Input  


def customer_organisation(departmentId,departmentName):
    Input =  {
              "departmentId": departmentId,
              "departmentName": departmentName
          }
    return Input  


def customer_requestType(requestType):
    Input =  {
              "requestType": requestType,
             }
    return Input     


def customer_location(country,stateProvince,zipPostCode,city):
    Input =  {
              "country": country,
              "stateProvince": stateProvince,
              "zipPostCode": zipPostCode,
              "city": city
          }
    return Input        


def customer_site(siteId,siteName,buildingId,buildingName):
    Input = {
              "siteId": siteId,
              "siteName": siteName,
              "buildingId": buildingId,
              "buildingName": buildingName
          }               
    return Input

def stationary_input(sector,fuelName,fuelAmount,fuelUnit,hvBasis):
    input = {"sector": sector,
             "fuelName": fuelName,
             "fuelAmount": fuelAmount,
             "fuelUnit": fuelUnit,
             "hvBasis": hvBasis
            }
    return input

### Stationary Emission Query - Example

Run the below snippet to set the request payload for Stationary Emission Endpoint Query

To check the accepted values for the individual fields of the request payload please visit:
https://developer.ibm.com/apis/catalog/envintelsuite--ibm-environmental-intelligence/GHG+Emissions+API#stationary

In [10]:
request_payload ={'customID'        : customer_id(customerid='6377709825425408'), 
                   'onBehalfOfClient': customer_onBehalfOfClient(companyId='1687653745426432',
                                                                 companyName='Mike Wright') , 
                   'organisation'    : customer_organisation(departmentId='1551140483235840',
                                                             departmentName='Cordelia McDaniel'), 
                   'requestType'     : customer_requestType(requestType='ACTUAL'), 
                   'location'        : customer_location(country='Niger',
                                                         stateProvince='QC',
                                                         zipPostCode='E1G 8B1',
                                                         city='Caposoode'),
                   'site'            : customer_site(siteId='5220563278626816',
                                                     siteName='Johnny Soto',
                                                     buildingId='7334540951420928',
                                                     buildingName='Grace Ryan'), 
                   'timePeriod'      : {'year' : '2021', 
                                        'month': '1'},
                   'activityData' : stationary_input(sector     = 'Energy', 
                                                        fuelName   = 'Coal tar',
                                                        fuelAmount = 1.5,
                                                        fuelUnit   = 'metric ton',
                                                        hvBasis    = 'Not applicable')
                   }        
       
print("Requests payload is :",json.dumps(request_payload, indent=2))   


Requests payload is : {
  "customID": {
    "id": "6377709825425408"
  },
  "onBehalfOfClient": {
    "companyId": "1687653745426432",
    "companyName": "Mike Wright"
  },
  "organisation": {
    "departmentId": "1551140483235840",
    "departmentName": "Cordelia McDaniel"
  },
  "requestType": {
    "requestType": "ACTUAL"
  },
  "location": {
    "country": "Niger",
    "stateProvince": "QC",
    "zipPostCode": "E1G 8B1",
    "city": "Caposoode"
  },
  "site": {
    "siteId": "5220563278626816",
    "siteName": "Johnny Soto",
    "buildingId": "7334540951420928",
    "buildingName": "Grace Ryan"
  },
  "timePeriod": {
    "year": "2021",
    "month": "1"
  },
  "activityData": {
    "sector": "Energy",
    "fuelName": "Coal tar",
    "fuelAmount": 1.5,
    "fuelUnit": "metric ton",
    "hvBasis": "Not applicable"
  }
}


Execute the below code to Run the Stationary API



In [15]:
EI_API_ENDPOINT = f"{EI_BASE_URL}/v2/carbon/stationary"
CO2Response = Call_Carbon_API(EI_API_ENDPOINT , request_payload, mybearer_token)

df_res = pd.json_normalize(CO2Response)
df_styled = df_res.style.set_table_styles([{
    'selector': 'table',
    'props' : [
        ('display', 'block'),
        ('overflow-x', 'auto'),
        ('white-space', 'nowrap')
    ]
}])
display_summary(df_styled)

,fossilFuelCO2,biogenicCO2,CH4,N2O,CO2e,unitOfMeasurement,description
0,3.389400,0,0.000042,0.000063,3.407271,metric tonne,"Fuel Analysis Method is used. Emission factors are obtained from 2019 Refinement to the 2006 IPCC Guidelines for National Greenhouse Gas Inventories. GWP values are obtained from IPCC Fifth Assessment Report, 2014 (AR5)"


#### Output Explanation

<b>fossilFuelCO2 </b> -     Release of amounts of Carbon Dioxide due to burning of fossil fuels (metric tonne)


<b>biogenicCO2</b> -       Release of amounts of Carbon Dioxide due to combustion or decomposition of biomass and its derivatives (metric tonne)

<b>CH4 </b>-               Release of  amounts of  Methane (metric tonne)

<b>N2O </b>-               Release of  amounts of  Nitrous Oxide (metric tonne)

<b>CO2e</b> -              Release of amounts of total greenhouse gases emitted (metric tonne)

<b>unitOfMeasurement</b>-  Unit used to measure